In [36]:
import pandas as pd
import folium
import requests
import re

There are two main steps in this:
1. We request list of public transport stations reachable in given time from HERE https://developer.here.com/documentation/examples/rest/public_transit/isochrone-search, it returns list of stations with coordinates and minutes spent to reach it

2. We then request pedestrian isohrones from HERE https://developer.here.com/documentation/routing/dev_guide/topics/resource-calculate-isoline.html for the rest of seconds. 

For example, if we want overall time of trip being 15 minutes and it took us 10 minutes to reach certain station, we then have to draw a pedestrian isohrone for 5 minutes.

### importing data

In [37]:
# reading api keys
api_keys = pd.read_excel('../api_keys.xlsx')
api_keys.set_index('key_name', inplace=True)

# link for mapbox map as a underlay for folium
map_url = api_keys.loc['mapbox_map']['key']

# HERE Rest api key
here_api_key = api_keys.loc['here_rest_api']['key']

In [38]:
# center point coords
center_lat, center_lon = 51.507351, -0.127660

In [39]:
# HERE API url for public transport stops
URL_pt = 'https://transit.ls.hereapi.com/v3/isochrone.json'

### request to Here API for public transport stops reachable from a point

works  with https://developer.here.com/documentation/examples/rest/public_transit/isochrone-search


- isohrones are requested for `15 minutes`, 
- start point is  from `center_lat, center_lon`, 
- time of departure is set as `2020-06-25T09:30:00+01:00`- 9:30 am in London.

In [40]:
#time in minutes
max_time = 15

#start time
start_time = '2020-06-25T09:30:00+01:00'

In [41]:
# start point 
start_point = f"{center_lat:.6f},{center_lon:.6f}"

# request params
params = { 
        'apiKey': here_api_key,
        'center': start_point,
        'maxDur': max_time,
        'time': start_time,
            }

# response
response = requests.get(URL_pt, params=params)
response_json = response.json()

# result in json
results = response_json['Res']['Isochrone']['IsoDest']

In [42]:
# create dataframe out of json results
stations = pd.json_normalize(results)
stations = stations.join(pd.json_normalize(stations['Stn'].explode()))
stations = stations.drop('Stn', axis=1)

# converting duration to int
stations['duration'] = pd.to_numeric(stations['duration'].str.replace("[PTM]", ""))

# rest time in seconds plus ten extra seconds to get small polygons at the ends of isohrones
stations['rest_seconds'] = (max_time*60)-(stations['duration']*60)+10
stations

,duration,y,x,name,id,rest_seconds
0,0,51.507412,-0.127252,Charing Cross,420326683,910
1,1,51.507282,-0.126594,Trafalgar Square (Stop W),420321619,850
2,1,51.508121,-0.127381,Trafalgar Square (Stop C),420321615,850
3,1,51.507160,-0.126700,Trafalgar Square (Stop X),420321620,850
4,2,51.507629,-0.129779,Trafalgar Square (Stop A),420321613,790
...,...,...,...,...,...,...
385,15,51.498692,-0.131413,Gt Smith St Westminster Abbey (Stop V),420322513,10
386,15,51.500717,-0.119342,St Thomas' Hospital County Hall (Stop D),420311382,10
387,15,51.534126,-0.139103,Mornington Crescent,420325535,10
388,15,51.522816,-0.158074,Baker Street Station,420306356,10


### pedestrian isohrones with HERE API, output for folium

works  with https://developer.here.com/documentation/routing/dev_guide/topics/resource-calculate-isoline.html

- isohrones are requested for the rest of time in seconds, taken from `stations['rest_seconds']`, 
- start points are taken from `stations`, 
- mode is pedestrian, 
- resolution is maximized

In [43]:
# HERE API url for isolines
URL_iso = 'https://isoline.route.ls.hereapi.com/routing/7.2/calculateisoline.json'

In [44]:
# returns isohrone polygons in format suitable for folium map 

def isoline_polygon_for_folium(lat, lon, time):
    #start point of isoline
    start_point = f"geo!{lat:.6f},{lon:.6f}"
    # request params
    params = { 
        'apiKey': here_api_key,
        'start': start_point,
        'range': time,
        'rangetype': 'time',
        'mode': 'fastest;pedestrian',
        'departure': 'now',
        'singlecomponent': 'true',
        'resolution': 10,
        'maxpoints': 300,
            }
    # response
    response = requests.get(URL_iso, params=params)
    response_json = response.json()
    
    try:
        # resulting polygon
        polygon = response_json['response']['isoline'][0]['component'][0]['shape']

        # creating dataframe with lon lat columns out of result
        poly_df = pd.DataFrame(polygon)
        coordinates = poly_df[0].str.split(",", expand = True)
        coordinates.columns = ['lat', 'lon']
       
        # lists of cordinates
        lat = coordinates['lat'].astype('float').to_list()
        lon = coordinates['lon'].astype('float').to_list()

        # tuple for polygon
        points = []
        for lat, lon in zip(lat, lon):
            points.append(tuple([lat, lon]))

        return points

    except Exception as e:
        print("for point", start_point)
        print("result is", response_json)
        print("which raises", e)
        return ""

In [45]:
stations['isoline_folium'] = stations.apply(lambda stations: isoline_polygon_for_folium
                                        (stations['y'],stations['x'], stations['rest_seconds']), axis=1)

stations.head()

,duration,y,x,name,id,rest_seconds,isoline_folium
0,0,51.507412,-0.127252,Charing Cross,420326683,910,"[(51.5068245, -0.1381016), (51.5071678, -0.137..."
1,1,51.507282,-0.126594,Trafalgar Square (Stop W),420321619,850,"[(51.5061378, -0.1367283), (51.5066528, -0.136..."
2,1,51.508121,-0.127381,Trafalgar Square (Stop C),420321615,850,"[(51.5061378, -0.1374149), (51.5066528, -0.137..."
3,1,51.507160,-0.126700,Trafalgar Square (Stop X),420321620,850,"[(51.5061378, -0.1367283), (51.5066528, -0.136..."
4,2,51.507629,-0.129779,Trafalgar Square (Stop A),420321613,790,"[(51.5047646, -0.1394749), (51.5052795, -0.139..."


### folium map with results

In [46]:
# folium map
m = folium.Map(location=[center_lat, center_lon], tiles = map_url, zoom_start = 12, attr = 'Mapbox')

In [47]:
#add polygons of pedestrian isohrones to map

for isoline in stations['isoline_folium'].to_list():
    folium.Polygon(locations = isoline, fill_color = 'blue', opacity = 0.2).add_to(m)

# coordinates of stops
point_lat = stations['y'].to_list()
point_lon = stations['x'].to_list()

# add stops with minutes to map
for lat, lon, duration in zip(point_lat, point_lon, stations['duration']):
    folium.CircleMarker(location=[lat, lon], radius = 4, fill_color='red', 
                        color="gray", fill_opacity = 0.5, popup = duration).add_to(m)

m

In [48]:
# to export map to html
#m.save("pt_15min_here_isolines.html")

In [49]:
# export stations list to csv
#stations.to_csv('./stations_15_min.csv')